## Week 2 Exercise - Nigerian Financial Advisor

This notebook explores building a financial advisor chatbot for the Nigerian market.

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
load_dotenv(override=True)

In [ ]:

OLLAMA_BASE_URL = "http://localhost:11434/v1"
OPEN_ROUTER_BASE_URL = "https://openrouter.ai/api/v1"
OPEN_ROUTER_API_KEY = os.getenv('OPEN_ROUTER_API_KEY')

client = OpenAI(
    base_url=OPEN_ROUTER_BASE_URL,
    api_key=OPEN_ROUTER_API_KEY
)

MODELS = [
    "openai/gpt-4.1-mini",
    "anthropic/claude-sonnet-4.6",
    "google/gemini-3-flash-preview"
]

SYSTEM_PROMPT = """
You are a Financial Advisor for the Nigerian market. 
Your goal is to help users navigate inflation, currency devaluation, and wealth building.

STRICT RULES FOR DATA ACCURACY:
1. Whenever a user asks about interest rates, yields, or 'how much' they can earn on a specific platform, you MUST call the 'get_investment_rates' tool.
2. DO NOT hallucinate or guess interest rates. If the tool returns an error or the platform is missing, inform the user you don't have the live data for that specific app yet.
3. Always compare the rates against the current Nigerian inflation (approx. 27%+) to provide context on 'real' returns.
4. If the user asks about USD investments, highlight the 'usd_yield' and 'usd_support' fields from the tool.

SUPPORTED PLATFORMS:
- Piggyvest
- Cowrywise
- Risevest
- Vasrefil

TONE & STYLE:
- Professional, yet localized. Use terms like 'Naira', 'Kobo', 'Safenet', or 'Hedge'.
- Be proactive: If a rate is low, suggest looking at the USD options provided by the tool.
"""

In [ ]:
def chat(message, history, model=MODELS[0]):
    try:
        history = [{"role":h["role"], "content":h["content"]} for h in history]
        messages = [{"role": "system", "content": SYSTEM_PROMPT}] + history + [{"role": "user", "content": message}]
        response = client.chat.completions.create(model=model, messages=messages, stream=True)

        message = ""
        for chunk in response:
            if chunk.choices[0].delta.content:
                message += chunk.choices[0].delta.content
                yield message
    
    except Exception as e:
        print(f"Error: {e}")
        return f"An error occurred, please try again later."

with gr.Blocks() as ui:
    gr.Markdown("## 🇳🇬 Financial Advisor Chatbot")
    with gr.Row():
        model_dropdown = gr.Dropdown(choices=MODELS, value=MODELS[0], label="Model")

    gr.ChatInterface(
        fn=chat,
        additional_inputs=[model_dropdown],
        type="messages",
        examples=[
            ["Which apps support USD?", MODELS[0]],
            ["What is the current rate on Risevest?", MODELS[1]],
        ]
    )


ui.launch()

In [ ]:
### Using Tools with the Chat Function

To enhance the chatbot's capabilities, you can integrate external tools or functions. This allows the chatbot to fetch real-time data or perform specific actions, such as retrieving investment rates from Nigerian fintech platforms. The tool definitions and handler functions are provided in later cells, enabling the chat interface to call these tools when needed.

For example, the chatbot can use the `get_investment_rates` function to provide up-to-date savings and investment rates for platforms like Piggyvest, Cowrywise, Risevest and Vasrefil. This makes the chatbot more interactive and informative for users seeking financial advice.

In [ ]:
def get_investment_rates(platform_name: str):
    print(f"Tool called: get_investment_rates with platform_name={platform_name}")
    """Provides 2026 investment and savings rates for Nigerian Fintechs."""
    data = {
        "piggyvest": {
            "naira_savings": "5% - 22% (Safelock yields highest)",
            "usd_support": "Yes (Flex Dollar)",
            "usd_rate": "7% p.a."
        },
        "cowrywise": {
            "naira_savings": "18% - 22% (Mutual Funds/Treasury Bills)",
            "usd_support": "Yes (Dollar Mutual Funds)",
            "usd_rate": "Variable based on fund performance"
        },
        "risevest": {
            "naira_vault": "Up to 25% p.a.",
            "usd_support": "Primary Focus (US Stocks, Real Estate, Fixed Income)",
            "usd_rate": "10% (Fixed Income) to 15% (Real Estate) p.a."
        },
        "vasrefil": {
            "naira_savings": "Up to 30% p.a. (Vasrefil Vault)",
            "usd_support": "No"
        }
    }
    platform = platform_name.lower().strip()
    return json.dumps(data.get(platform, {"error": "Platform not found in database."}))

tools = [{
    "type": "function",
    "function": {
        "name": "get_investment_rates",
        "description": "Fetch current Naira and USD interest rates from the SQLite database.",
        "parameters": {
            "type": "object",
            "properties": {
                "platform_name": {"type": "string", "description": "The fintech app name (e.g. Piggyvest)"}
            },
            "required": ["platform_name"]
        }
    }
}]


In [ ]:
def handle_tool_calls(message):
    print(f"Handling tool calls for message: {message}")
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_investment_rates":
            arguments = json.loads(tool_call.function.arguments)
            platform_name = arguments.get('platform_name')
            price_details = get_investment_rates(platform_name)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
    return responses

In [ ]:
def chat_with_tools(message, history, model=MODELS[0]):
    try:
        history = [{"role":h["role"], "content":h["content"]} for h in history]
        messages = [{"role": "system", "content": SYSTEM_PROMPT}] + history + [{"role": "user", "content": message}]
        response = client.chat.completions.create(model=model, messages=messages, tools=tools)
        
        
        if response.choices[0].finish_reason=="tool_calls":
            message = response.choices[0].message
            responses = handle_tool_calls(message)
            messages.append(message)
            messages.extend(responses)
            response = client.chat.completions.create(model=model, messages=messages, tools=tools, stream=True)
            message = ""
            for chunk in response:
                if chunk.choices[0].delta.content:
                    message += chunk.choices[0].delta.content
                    yield message
        else:
            response = client.chat.completions.create(model=model, messages=messages, stream=True)
            message = ""
            for chunk in response:
                if chunk.choices[0].delta.content:
                    message += chunk.choices[0].delta.content
                    yield message

    except Exception as e:
        print(f"Error: {e}")
        return f"An error occurred, please try again later."

with gr.Blocks() as ui:
    gr.Markdown("## 🇳🇬 Financial Advisor Chatbot")
    with gr.Row():
        model_dropdown = gr.Dropdown(choices=MODELS, value=MODELS[0], label="Model")

    gr.ChatInterface(
        fn=chat_with_tools,
        additional_inputs=[model_dropdown],
        type="messages",
        examples=[
            ["Which apps support USD?", MODELS[0]],
            ["What is the current rate on Risevest?", MODELS[1]],
        ]
    )

ui.launch()